In [28]:
import json
import ee
import geopandas as gpd
import os
import zipfile

from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import shape
from datetime import datetime


In [19]:
# This will open a link for you to log into your Google account
ee.Authenticate()

# Initialize Earth Engine
ee.Initialize()

print("Earth Engine is ready to use!")

Enter verification code:  4/1AVMBsJg5C_CZYYSe90tAgLfDdhJ1nx5u50lt3tmCEpAvT8QUcKYiLiUTBXs



Successfully saved authorization token.
Earth Engine is ready to use!


In [24]:
# geojson_path = "./rice_practices/mask_ready/partial_labels_grids_FID.geojson"
# gdf = gpd.read_file(geojson_path)

# shapefile_folder = "./rice_practices/mask_ready/partial_labels_grids_FID_shp"
# os.makedirs(shapefile_folder, exist_ok=True)
# shapefile_path = os.path.join(shapefile_folder, "partial_labels_grids_FID.shp")
# gdf.to_file(shapefile_path)

# # Step 3: Zip the shapefile components
# zip_path = "./rice_practices/mask_ready/partial_labels_grids_shp_FID.zip"
# with zipfile.ZipFile(zip_path, 'w') as zipf:
#     for file_name in os.listdir(shapefile_folder):
#         file_path = os.path.join(shapefile_folder, file_name)
#         zipf.write(file_path, arcname=file_name)

# print(f"Shapefile zipped at: {zip_path}")

Shapefile zipped at: ./rice_practices/mask_ready/partial_labels_grids_shp_FID.zip


## Download S1

In [25]:
# Load the grid collection
grids = ee.FeatureCollection("projects/ee-rahebeabedi2000/assets/partial_labels_grids_shp_FID")
# Check property names of the first feature
first_feature = ee.Feature(grids.first())
property_names = first_feature.propertyNames().getInfo()

print("Property names in first grid feature:", property_names)


Property names in first grid feature: ['FID', 'system:index', 'filename']


In [ ]:
# Config
scale = 10  # meters per pixel
pixel_count = 224
export_size = scale * pixel_count  # 2240 meters
drive_folder = 'sentinel_1_partial_labels'
export_log_path = "exported_tiles.txt"
geojson_output_path = "exported_regions.geojson"

In [32]:
grid_list = grids.toList(grids.size())

# Sentinel-1 collection
base_s1 = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
)

# Load already exported names
try:
    with open(export_log_path, "r") as f:
        already_exported = set(line.strip() for line in f if line.strip())
except FileNotFoundError:
    already_exported = set()

# Metadata collector
export_metadata = []

# Export function
def export_grid(i):
    global export_metadata
    try:
        grid_feature = ee.Feature(grid_list.get(i))

        # Get FID directly from property
        fid_str = grid_feature.get("FID").getInfo()  # e.g., tile000000

        # Extract date from filename
        filename = grid_feature.get("filename").getInfo()  # e.g., '2024_07/raster_07_158670.tif'
        yyyymm = filename.split("/")[0]  # → '2024_07'
        year, month = yyyymm.split("_")
        start_date = f"{year}-{month}-01"
        end_date = f"{year}-{month}-30"

        # Export name format
        export_name = f"{fid_str}_{yyyymm}_s1"
        if export_name in already_exported:
            print(f"Skipping {export_name} — already exported")
            return

        # Geometry and CRS
        centroid = grid_feature.geometry().centroid()
        crs = "EPSG:32644"  # Replace with dynamic zone if needed

        # Filter Sentinel-1 image
        filtered = base_s1.filterDate(start_date, end_date).filterBounds(centroid)
        img = filtered.first()
        if img is None or img.getInfo() is None:
            print(f"No S1 image for {yyyymm} — Grid {fid_str}")
            return

        sentinel_image = ee.Image(img)

        renamed = sentinel_image.select(['VV', 'VH', 'angle'], [
            f"{yyyymm}_VV", f"{yyyymm}_VH", f"{yyyymm}_angle"
        ]).toFloat()

        export_region = centroid.buffer(export_size / 2).bounds()

        # Export task
        task = ee.batch.Export.image.toDrive(
            image=renamed,
            description=export_name,
            folder=drive_folder,
            fileNamePrefix=export_name,
            region=export_region,
            crs=crs,
            scale=scale,
            maxPixels=1e9
        )
        task.start()

        # Log
        with open(export_log_path, "a") as f:
            f.write(export_name + "\n")

        geometry_shape = shape(export_region.getInfo())
        export_metadata.append({
            "export_name": export_name,
            "fid": fid_str,
            "geometry": geometry_shape,
            "timestamp": datetime.utcnow().isoformat() + "Z"
        })

        print(f"Exporting {export_name}")

    except Exception as e:
        print(f"Error on grid {i}: {e}")

# Parallel export with limited threads
total_grids = grids.size().getInfo()
with ThreadPoolExecutor(max_workers=3) as executor:
    list(tqdm(executor.map(export_grid, range(total_grids)), total=total_grids))

# Save GeoJSON metadata
if export_metadata:
    gdf = gpd.GeoDataFrame(export_metadata, geometry="geometry", crs="EPSG:4326")
    gdf.to_file(geojson_output_path, driver="GeoJSON")
    print(f"GeoJSON saved to {geojson_output_path}")
else:
    print("No new exports to record")


  0%|                                          | 1/735 [00:00<05:05,  2.40it/s]

Skipping tile000000_2024_07_s1 — already exported


  0%|                                          | 2/735 [00:01<07:19,  1.67it/s]

Exporting tile000000_2024_09_s1
Exporting tile000000_2024_11_s1
Skipping tile000001_2024_07_s1 — already exported


  1%|▏                                         | 4/735 [00:01<05:06,  2.38it/s]

...

100%|███████████████████████████████████████▉| 733/735 [03:29<00:00,  3.80it/s]

Exporting tile000222_2024_08_s1
Exporting tile000223_2024_08_s1


100%|████████████████████████████████████████| 735/735 [03:29<00:00,  3.50it/s]

Exporting tile000224_2024_08_s1


GeoJSON saved to exported_regions.geojson


### canceling a task

In [ ]:
# List all tasks
tasks = ee.batch.Task.list()

# Cancel all tasks that are either pending or running
for t in tasks:
    if t.state in ['READY', 'RUNNING']:
        print(f"Cancelling task: {t.config.get('description')} (ID: {t.id})")
        t.cancel()